# KNN_Confusions.ipynb 

In [ ]:
###############
## Libraries ##
###############

import tensorflow as tf
import matplotlib.pyplot as plt 
import numpy as np
from tensorflow.keras import datasets, layers, models, losses
from keras.callbacks import ModelCheckpoint
from keras.models import load_model

In [ ]:
# load in mnist digit data
all_data = np.load("/scratch/gpfs/eysu/src_data/mnist.npz")
print(all_data.files)
x_test = all_data['x_test']
x_train = all_data['x_train']
y_train = all_data['y_train']
y_test = all_data['y_test']

# Split data and reset dimensions
labels = ["0",  # index 0
          "1",  # index 1
          "2",  # index 2 
          "3",  # index 3 
          "4",  # index 4
          "5",  # index 5
          "6",  # index 6 
          "7",  # index 7 
          "8",  # index 8 
          "9"]  # index 9

# Print training set shape - note there are 60,000 training data of image size of 28x28, 60,000 train labels)
print("x_train shape:", x_train.shape, "y_train shape:", y_train.shape)

# save train labels
y_train_labels = y_train
y_test_labels = y_test

# Print the number of training and test datasets
print(x_train.shape[0], 'train set')
print(x_test.shape[0], 'test set')


x_train = x_train.astype('float32') / 255
x_test = x_test.astype('float32') / 255

# Further break training data into train / validation sets (# put 5000 into validation set and keep remaining 55,000 for train)
(x_train, x_valid) = x_train[5000:], x_train[:5000] 
(y_train, y_valid) = y_train[5000:], y_train[:5000]

# Reshape input data from (28, 28) to (28, 28, 1)
w, h = 28, 28
x_train = x_train.reshape(x_train.shape[0], w, h, 1)
x_valid = x_valid.reshape(x_valid.shape[0], w, h, 1)
x_test = x_test.reshape(x_test.shape[0], w, h, 1)

# Validation set
y_valid = tf.keras.utils.to_categorical(y_valid, 10)
y_test = tf.keras.utils.to_categorical(y_test, 10)
    
# Image index, you can pick any number between 0 and 59,999
img_index = 5
# y_train contains the lables, ranging from 0 to 9
label_index = y_train[img_index]
# Print the label, for example 2 Pullover
print ("y = " + str(label_index) + " " +(labels[label_index]))
# # Show one of the images from the training dataset
plt.imshow(x_train[img_index])
plt.show()

In [ ]:
################################################
## Recreate model from final iterations of SR ##
################################################

# specify path of model to load in
model_weights = 'Serial-Reproductions-CNN-Research/weights_concise/weights_digits_1_trunc/model.weights.best.iter24.hdf5'

# load model
model = load_model(model_weights)

# examine model
model.summary()
model.get_weights()
model.optimizer

layer_output = model.layers[-2].output
activation_model = models.Model(inputs=model.input, outputs=layer_output)

# Run image through model and get activations
activations = activation_model.predict(x_train) # should be 2 numpy arrays of dimension N images x 256D

print(activations.shape)

In [ ]:
###########################################################################################
## Read prediction softmax layer activations for all test set images, and all iterations ##
###########################################################################################
DATASET = '_DigitMNIST' 
REGIME = '_TRAINED_'

MAX_ITER = 25

# Build arrays of dimensions: N training images X L labels X P iterations
y_hat_train_arr = np.zeros([y_train.shape[0], len(labels), MAX_ITER])
y_hat_test_arr = np.zeros([y_test.shape[0], len(labels), MAX_ITER])

for i in range(MAX_ITER):
    if i == 0:
        y_hat_train_name = 'y_hat_train_seed'
        y_hat_test_name = 'y_hat_test_seed'
    
    else:
        y_hat_train_name = 'y_hat_train_' + 'iter' + str(i)
        y_hat_test_name = 'y_hat_test_' + 'iter' + str(i)
        
    # Load test set softmax outputs 
    yhtr = np.load('/scratch/gpfs/eysu/mock_supervised_weights/' + y_hat_train_name + '.npy')
    yhte = np.load('/scratch/gpfs/eysu/mock_supervised_weights/' + y_hat_test_name + '.npy')  

    # The first time through, use binary weight vectors to save correct class array
    if i == 0:
        true_class_tr = np.nonzero(yhtr)[1]
        true_class_te = np.nonzero(yhte)[1]
        
    y_hat_train_arr[:, :, i] = yhtr
    y_hat_test_arr[:, :, i] = yhte
    
    final_predictions = np.argmax(y_hat_train_arr[:,:, 24], axis=1)


print(y_hat_train_arr.shape)
# (55000, 50, 10)
print(y_hat_test_arr.shape)
# (10000, 50, 10)
print(final_predictions.shape)
# (55000)


In [ ]:
############################
## Load Nearest Neighbors ##
############################

# code to import nearest neighbors matrices
import pandas as pd
# read in 55000 x 25 matrices of 25 nearest in class neighbors and their norms

# digits
nearest_neighbors_in_class = pd.read_csv("Serial-Reproductions-CNN-Research/Outputs/NearestNeighbors/digits/nearest_neighbors_in_class_iter1.csv", sep = ',', header=None).to_numpy()
nearest_neighbors_in_class_norms = pd.read_csv("Serial-Reproductions-CNN-Research/Outputs/NearestNeighbors/digits/nearest_neighbors_in_class_norms_iter1.csv", sep = ',', header=None).to_numpy()

nearest_neighbors_other_class = pd.read_csv("Serial-Reproductions-CNN-Research/Outputs/NearestNeighbors/digits/nearest_neighbors_other_class_iter1.csv", sep = ',', header=None).to_numpy()
nearest_neighbors_other_class_norms = pd.read_csv("Serial-Reproductions-CNN-Research/Outputs/NearestNeighbors/digits/nearest_neighbors_other_class_norms_iter1.csv", sep = ',', header=None).to_numpy()

# fashion
# nearest_neighbors_in_class = pd.read_csv("Outputs/NearestNeighbors/fashion/nearest_neighbors_in_class.csv", sep = ',', header=None).to_numpy()
# nearest_neighbors_in_class_norms = pd.read_csv("Outputs/NearestNeighbors/fashion/nearest_neighbors_in_class_norms.csv", sep = ',', header=None).to_numpy()

# nearest_neighbors_other_class = pd.read_csv("Outputs/NearestNeighbors/fashion/nearest_neighbors_other_class.csv", sep = ',', header=None).to_numpy()
# nearest_neighbors_other_class_norms = pd.read_csv("Outputs/NearestNeighbors/fashion/nearest_neighbors_other_class_norms.csv", sep = ',', header=None).to_numpy()

print(nearest_neighbors_in_class.shape)
print(nearest_neighbors_in_class)

In [ ]:
####################################################
## Helper Function Make 9XN plots of 4 nearest in ##
## and out of class neighbors for given image     ##
####################################################


def helper_plot_4NN(image_idx):
    # plot image in center with 4 least confusing in and out of class image alongside
    figure = plt.figure(figsize=(20, 8), constrained_layout=True)
    spec = figure.add_gridspec(ncols=9, nrows=1)

    ax1 = figure.add_subplot(spec[0,0], xticks=[], yticks=[])
    im1 = ax1.imshow(x_train[nearest_neighbors_in_class[image_idx, 3]], cmap='gray')
    ax1.set_title("Class: " + str(labels[y_train[nearest_neighbors_in_class[image_idx, 3]]]) + 
                  "\nNorm diff: " + str(np.around(nearest_neighbors_in_class_norms[image_idx, 3], 3)) +
                  "\nFinal predicted class: " + str(labels[final_predictions[nearest_neighbors_in_class[image_idx, 3]]]), color = 'green')

    ax2 = figure.add_subplot(spec[0,1], xticks=[], yticks=[])
    im2 = ax2.imshow(x_train[nearest_neighbors_in_class[image_idx, 2]], cmap='gray')
    ax2.set_title("Class: " + str(labels[y_train[nearest_neighbors_in_class[image_idx, 2]]]) + 
                  "\nNorm diff: " + str(np.around(nearest_neighbors_in_class_norms[image_idx, 2], 3)) +
                  "\nFinal predicted class: " + str(labels[final_predictions[nearest_neighbors_in_class[image_idx, 2]]]), color = 'green')
    
    ax3 = figure.add_subplot(spec[0,2], xticks=[], yticks=[])
    im3 = ax3.imshow(x_train[nearest_neighbors_in_class[image_idx, 1]], cmap='gray')
    ax3.set_title("Class: " + str(labels[y_train[nearest_neighbors_in_class[image_idx, 1]]]) + 
                  "\nNorm diff: " + str(np.around(nearest_neighbors_in_class_norms[image_idx, 1], 3)) +
                  "\nFinal predicted class: " + str(labels[final_predictions[nearest_neighbors_in_class[image_idx, 1]]]), color = 'green')

    ax4 = figure.add_subplot(spec[0,3], xticks=[], yticks=[])
    im4 = ax4.imshow(x_train[nearest_neighbors_in_class[image_idx, 0]], cmap='gray')
    ax4.set_title("Class: " + str(labels[y_train[nearest_neighbors_in_class[image_idx, 0]]]) + 
                  "\nNorm diff: " + str(np.around(nearest_neighbors_in_class_norms[image_idx, 0], 3)) +
                  "\nFinal predicted class: " + str(labels[final_predictions[nearest_neighbors_in_class[image_idx, 0]]]), color = 'green')

    ax5 = figure.add_subplot(spec[0,4], xticks=[], yticks=[])
    im5 = ax5.imshow(x_train[image_idx], cmap='gray')
    ax5.set_title("Class: " + str(labels[y_train[image_idx]]) + "\nImage" + 
                  "\nFinal predicted class: " + str(labels[final_predictions[image_idx]]), color = 'green')


    ax6 = figure.add_subplot(spec[0,5], xticks=[], yticks=[])
    im6 = ax6.imshow(x_train[nearest_neighbors_other_class[image_idx, 0]], cmap='gray')
    ax6.set_title("Class: " + str(labels[y_train[nearest_neighbors_other_class[image_idx, 0]]]) + 
                  "\nNorm diff: " + str(np.around(nearest_neighbors_other_class_norms[image_idx, 0], 3)) +
                  "\nFinal predicted class: " + str(labels[final_predictions[nearest_neighbors_other_class[image_idx, 0]]]), color = 'red')
    
    ax7 = figure.add_subplot(spec[0,6], xticks=[], yticks=[])
    im7 = ax7.imshow(x_train[nearest_neighbors_other_class[image_idx, 1]], cmap='gray')
    ax7.set_title("Class: " + str(labels[y_train[nearest_neighbors_other_class[image_idx, 1]]]) + 
                  "\nNorm diff: " + str(np.around(nearest_neighbors_other_class_norms[image_idx, 1], 3)) +
                  "\nFinal predicted class: " + str(labels[final_predictions[nearest_neighbors_other_class[image_idx, 1]]]), color = 'red')
  
                  
    ax8 = figure.add_subplot(spec[0,7], xticks=[], yticks=[])
    im8 = ax8.imshow(x_train[nearest_neighbors_other_class[image_idx, 2]], cmap='gray')
    ax8.set_title("Class: " + str(labels[y_train[nearest_neighbors_other_class[image_idx, 2]]]) + 
                  "\nNorm diff: " + str(np.around(nearest_neighbors_other_class_norms[image_idx, 2], 3)) +
                  "\nFinal predicted class: " + str(labels[final_predictions[nearest_neighbors_other_class[image_idx, 2]]]), color = 'red')
   

    ax9 = figure.add_subplot(spec[0,8], xticks=[], yticks=[])
    im9 = ax9.imshow(x_train[nearest_neighbors_other_class[image_idx, 3]], cmap='gray')
    ax9.set_title("Class: " + str(labels[y_train[nearest_neighbors_other_class[image_idx, 3]]]) + 
                  "\nNorm diff: " + str(np.around(nearest_neighbors_other_class_norms[image_idx, 3], 3)) +
                  "\nFinal predicted class: " + str(labels[final_predictions[nearest_neighbors_other_class[image_idx, 3]]]), color = 'red')

    plt.show()
    return figure
    
    

In [ ]:
###################################
## Helper Function for 4xN plots ##
###################################

def print_images_helper(index, softmax, image, title):
    figure = plt.figure(figsize=(40, 40))
    # plot image
    ax1 = figure.add_subplot(8, 8, 1, xticks=[], yticks=[])
    im1 = ax1.imshow(image, cmap='gray')
    ax1.set_title(title)

    # plot weights graph
    if (softmax[:, 0].all() == softmax[:, 1].all()):
        title_str = "Softmax Outputs (iter 1 same)"
        
    else: title_str = "Softmax Outputs"
        
    ax2 = figure.add_subplot(8, 8, 2)
    im2 = ax2.imshow(softmax.T, cmap='Wistia')
    
    divider = make_axes_locatable(ax2)
    cax = divider.append_axes('right', size='5%', pad=0.05)
    cbar = figure.colorbar(im2, cax=cax, ticks=[0, 1])
    cbar.ax.set_yticklabels(['0', '1'])

    ax2.set(xlabel='Classes', ylabel='Iterations', title=title_str)
    ax2.set_xticks(ticks=[0, 1, 2, 3, 4, 5, 6, 7, 8, 9])
    ax2.set_xticklabels(labels, rotation = 'vertical')

    # plot correlation graph
    corr_arr = np.corrcoef(softmax.T)

    ax3 = figure.add_subplot(8, 8, 3)
    im3 = ax3.imshow(corr_arr, cmap='Wistia')
    divider = make_axes_locatable(ax3)
    cax = divider.append_axes('right', size='5%', pad=0.05)
    cbar = figure.colorbar(im2, cax=cax, orientation='vertical', ticks=[0, 1])
    cbar.ax.set_yticklabels(['0', '1'])
    ax3.set_title("Correlation Matrix")

    # plot eigenvalues graph
    eigs, _ = np.linalg.eig(corr_arr)
    num_nonzero = np.count_nonzero(np.around(eigs, 2))

    if (num_nonzero == 1):
        title_str = "Sorted Eigenvalues (" + str(num_nonzero) + " nonzero)"
    else:
        title_str = "Sorted Eigenvalues (" + str(num_nonzero) + " nonzeros)"

    ax4 = figure.add_subplot(8, 8, 4)
    im4 = ax4.plot(eigs, marker='o')
    ax4.set(xlabel="PC Number", xlim=[0,len(eigs)], ylim=[0,MAX_ITER], title=title_str)

    plt.show()
    return figure
    

In [ ]:
########################
## Visualize N images ##
########################

from mpl_toolkits.axes_grid1 import make_axes_locatable
import warnings
warnings.filterwarnings('ignore')
import matplotlib.backends.backend_pdf

def visualize_N_images(CLASS, NUM_IMAGES, rank):
    print("Class: ", CLASS)
    # least confusing
    
    pdf = matplotlib.backends.backend_pdf.PdfPages("Serial-Reproductions-CNN-Research/NearestNeighborsUpdate/Iter1/least_confusing_" + str(labels[CLASS]) + "s.pdf")
    print(str(NUM_IMAGES) + " least confusing " + str(labels[CLASS]) + "s")
    for i in range(NUM_IMAGES):
        # get index of image
        image_idx = rank[i]
        
        # print image itself
        figure = print_images_helper(image_idx, y_hat_train_arr[image_idx], x_train[image_idx], "Original Image")
        pdf.savefig(figure, bbox_inches='tight')

        figure2 = helper_plot_4NN(image_idx)
        pdf.savefig(figure2, bbox_inches='tight')
        
    pdf.close()

    # most confusing
    
    pdf = matplotlib.backends.backend_pdf.PdfPages("Serial-Reproductions-CNN-Research/NearestNeighborsUpdate/Iter1/most_confusing_" + str(labels[CLASS]) + "s.pdf")
    print(str(NUM_IMAGES) + " most confusing " + str(labels[CLASS]) + "s")
    for i in range(NUM_IMAGES):
        # get index of image
        image_idx = rank[-NUM_IMAGES + i]
        
        # print image itself
        figure = print_images_helper(image_idx, y_hat_train_arr[image_idx], x_train[image_idx], "Original Image")
        pdf.savefig(figure, bbox_inches='tight')

        figure2 = helper_plot_4NN(image_idx)
        pdf.savefig(figure2, bbox_inches='tight')
        
    pdf.close()

In [ ]:
##############################################
## Rank by difference to binary seed vector ##
##############################################

# Call visualization methods and generate PDFs from here

for i in range(10):
    # choose which class to analyze
    CLASS = i
    same_class_idxs = np.array(np.where(y_train == CLASS)).squeeze()
    class_data = y_hat_train_arr[np.where(true_class_tr == CLASS), :, :].squeeze()
    diff_arr = class_data[:, CLASS, 0] - class_data[:, CLASS, MAX_ITER - 1]
    seed_diff_rank = np.argsort(diff_arr)
    
    # again, convert subarray indices back to master indices
    seed_diff_rank = same_class_idxs[seed_diff_rank]
    print(seed_diff_rank)
    visualize_N_images(i, 50, seed_diff_rank)